In [2]:
# 1. Import libraries
import pandas as pd

# 2. Load & check data
df = pd.read_csv('marketing_campaign.csv', sep='\t')
print("Dataset shape:", df.shape)
print("\nMissing values:\n", df.isnull().sum())

# 3. Data Cleaning

# 3.1 Remove rows with missing 'Income'
df = df.dropna(subset=['Income'])

# 3.2 Convert 'Dt_Customer' column to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)

# 3.3 Create 'TotalChildren' column
df['TotalChildren'] = df['Kidhome'] + df['Teenhome']

# 3.4 Calculate 'TotalSpent' column
df['TotalSpent'] = (
    df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] +
    df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']
)

# 4. Calculate RFM metrics
# R = Recency (days since last purchase)
snapshot_date = df['Dt_Customer'].max()  # or use a fixed reference date
df['Recency'] = (snapshot_date - df['Dt_Customer']).dt.days

# F = Frequency (total number of purchases)
df['Frequency'] = (
    df['NumDealsPurchases'] + df['NumWebPurchases'] +
    df['NumCatalogPurchases'] + df['NumStorePurchases']
)
# M = Monetary (total spending amount)
df['Monetary'] = df['TotalSpent']

# 5. Calculate RFM score
df['R_Score'] = pd.qcut(df['Recency'], 4, labels=[4, 3, 2, 1]).astype(int)  # lower recency = higher score
df['F_Score'] = pd.qcut(df['Frequency'], 4, labels=[1, 2, 3, 4]).astype(int)
df['M_Score'] = pd.qcut(df['Monetary'], 4, labels=[1, 2, 3, 4]).astype(int)

df['RFM_Score'] = df['R_Score'] + df['F_Score'] + df['M_Score']

# 6. Segment customers
def segment_me(rfm_score):
    if rfm_score >= 9:
        return 'VIP'
    elif rfm_score >= 7:
        return 'Loyal'
    elif rfm_score >= 5:
        return 'Potential'
    else:
        return 'At Risk'

df['Segment'] = df['RFM_Score'].apply(segment_me)

# 7. Display customers in each segment
for seg in df['Segment'].unique():
    segment_customers = df[df['Segment'] == seg]
    print(f"\n===== Segment: {seg} =====")
    print(f"Number of customers: {segment_customers.shape[0]}")
    print("Customer IDs:")
    print(segment_customers['ID'].values)

# 8. Business Suggestions
suggestions = {
    'VIP': "Maintain relationship, offer exclusive deals, invite to VIP events.",
    'Loyal': "Provide periodic vouchers, loyalty programs to maintain engagement.",
    'Potential': "Send special promotions to encourage more purchases.",
    'At Risk': "Send strong discount offers, survey reasons for decreased activity."
}

print("\nBusiness Suggestions:")
for seg, sug in suggestions.items():
    print(f"{seg}: {sug}")

Dataset shape: (2240, 29)

Missing values:
 ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

===== Segment: VIP =====
Number of customers: 803
Customer IDs:
[ 5524  4141  5324  7446  2125  2114  6565  5376  4047  1966  8755  4339
  2968  8601  1371  6566  8614  4114  2225  9381  6